In [3]:
import docs

github_data = docs.read_github_data()
parsed_data = docs.parse_data(github_data)
chunks = docs.chunk_documents(parsed_data)

In [9]:
from minsearch import Index

index = Index(
    text_fields=["content", "filename", "title", "description"],
)

index.fit(chunks)

In [10]:
from typing import Any, Dict, List, TypedDict

class SearchResult(TypedDict):
    """Represents a single search result entry."""
    start: int
    content: str
    title: str
    description: str
    filename: str


def search(query: str) -> List[SearchResult]:
    """
    Search the index for documents matching the given query.

    Args:
        query (str): The search query string.

    Returns:
        List[SearchResult]: A list of search results. Each result dictionary contains:
            - start (int): The starting position or offset within the source file.
            - content (str): A text excerpt or snippet containing the match.
            - title (str): The title of the matched document.
            - description (str): A short description of the document.
            - filename (str): The path or name of the source file.
    """
    return index.search(
        query=query,
        num_results=5,
    )

In [11]:
file_index = {}

for item in parsed_data:
    filename = item['filename']
    content = item['content']
    file_index[filename] = content

In [12]:
len(file_index)

95

In [13]:
def read_file(filename: str) -> str:
    """
    Retrieve the contents of a file from the file index if it exists.

    Args:
        filename (str): The name of the file to read.

    Returns:
        str: The file's contents if found, otherwise an error message 
        indicating that the file does not exist.
    """
    if filename in file_index:
        return file_index[filename]
    return "File doesn't exist"

In [18]:
documentation_agent_instructions = """
You are an assistant that helps improve and generate high-quality documentation for the project.

You have access to the following tools:
- search — Use this to explore topics in depth. Make multiple search calls if needed to gather comprehensive information.
- read_file — Use this when code snippets are missing or when you need to retrieve the full content of a file for context.

If `read_file` cannot be used or the file content is unavailable, clearly state:
> "Unable to verify with read_file."

When answering a question:
1. Provide file references for all source materials.  
   Use this format:  
   [{filename}](https://github.com/evidentlyai/docs/blob/main/{filename})
2. If the topic is covered in multiple documents, cite all relevant sources.
3. Include code examples whenever they clarify or demonstrate the concept.
4. Be concise, accurate, and helpful — focus on clarity and usability for developers.
5. If documentation is missing or unclear, infer from context and note that explicitly.

Example Citation:
See the full implementation in [metrics/api_reference.md](https://github.com/evidentlyai/docs/blob/main/metrics/api_reference.md).
""".strip()

In [19]:
from pydantic_ai import Agent

In [20]:
documentation_agent = Agent(
    name='documentation_agent',
    instructions=documentation_agent_instructions,
    tools=[search, read_file],
    model='openai:gpt-4o-mini'
)

In [29]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import PydanticAIRunner

In [30]:
runner = PydanticAIRunner(
    chat_interface=IPythonChatInterface(),
    agent=documentation_agent
)

https://github.com/alexeygrigorev/toyaikit/blob/main/toyaikit/chat/runners.py

In [31]:
await runner.run();

You: how do I run llm as a judge evals


You: stop


Chat ended.


In [21]:
results = await documentation_agent.run(
    user_prompt='how do I run llm as a judge evals',
    # message_history=results.all_messages()
)

In [28]:
for message in results.new_messages():
    print(message.kind)

    for part in message.parts:
        print(part.part_kind)

    print()

request
user-prompt

response
tool-call

request
tool-return

response
text

